In [ ]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

In [2]:
from libnlp import preprocessing
import pandas as pd

In [3]:
train_path='../Data/train.csv'
test_path='../Data/test.csv'

In [4]:
train=pd.read_csv(train_path,encoding='utf-8')

In [5]:
train.head(10)

,ID,text,label
0,13P0QT0,3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...,-1
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
3,U0TTYY8,ak slouma,1
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1
5,OSTX8NX,hhhhhhhh blidaa minik ba3d doussieet athika il...,1
6,I1JT3SJ,wahdek big boss,1
7,54FC0NS,9arwwwwii yhbb 3alaa bouwahh la7niynn 5hndhawii,-1
8,0NF40PI,ti blaad faasda ya hsouna hedheka el wa9a3 b a...,-1
9,2ZB3IIF,3omra ma9boula si slim,1


In [6]:
tweet=train[train.ID=='IFOGGPJ'].text.values

In [7]:
tweet[0]

'chhkrathommmm ch3andkom ménnouu fi chkoun testablhouu yé5iii baaaarcha niféé999 w 7étta li 3omrouu 5 snin wala yéfhémm ma5yébkommmmm'

In [8]:
preprocessing.preprocess('chhkrathomm Ch3andkom JAMIU ménnouu fi chkoun testablhouu yé5ii baarcha niféé9 w 7étta li 3omrouu snin wala yéfhémm ma5yébkomm')

'chhkrathomm ch3andkom jamiu ménnouu fi chkoun testablhouu yé5ii baarcha niféé9 w 7étta li 3omrouu snin wala yéfhémm ma5yébkomm'

In [9]:
train['text']=train.text.apply(lambda x: preprocessing.preprocess(x))

In [10]:
train.head()

,ID,text,label
0,13P0QT0,3sbaa lek ou le seim riahi ou 3sbaa le ca,-1
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
3,U0TTYY8,ak slouma,1
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1


In [11]:
train.drop(['ID'],axis=1,inplace=True)

In [12]:
train.to_csv('train_ppd.csv',index=False)

In [13]:
test=pd.read_csv(test_path,encoding='utf-8')

In [14]:
test.head()

,ID,text
0,2DDHQW9,barcha aaindou fiha hak w barcha teflim kadhalik
1,5HY6UEY,ye gernabou ye 9a7ba
2,ATNVUJX,saber w barra rabbi m3ak 5ouya
3,Q9XYVOQ,cha3ébbb ta7aaaaannnnnnnnnnn tfouuhh
4,TOAHLRH,rabi y5alihoulek w yfar7ek bih w inchallah itc...


In [15]:
import numpy as np
ls1=list(np.arange(-1,2))

In [16]:
dict([(key,val) for val,key in enumerate(ls1)])

{-1: 0, 0: 1, 1: 2}

In [17]:
ls1=set(train.label)

In [18]:
ls1

{-1, 0, 1}

In [19]:
dict([(key,val) for val,key in enumerate(ls1)])

{0: 0, 1: 1, -1: 2}

In [20]:
csv_p='train.csv'
f_name=csv_p.split('.')

In [21]:
f_name

['train', 'csv']

# Creating Torch text data

In [22]:
import torchtext.data as ttd
import torch
#Definining the tokenizer
tokenizer=torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-multilingual-uncased',)
pad_token=tokenizer.pad_token
unk_token=tokenizer.unk_token

Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master


In [23]:
tokenizer.tokenize(
'chhkrathomm ch3andkom ménnouu fi chkoun testablhouu yé5ii baarcha niféé9 w 7étta li 3omrouu snin wala yéfhémm ma5yébkomm')

['ch',
 '##h',
 '##krat',
 '##hom',
 '##m',
 'ch',
 '##3',
 '##and',
 '##kom',
 'menn',
 '##ou',
 '##u',
 'fi',
 'ch',
 '##kou',
 '##n',
 'testa',
 '##bl',
 '##hou',
 '##u',
 'ye',
 '##5',
 '##ii',
 'ba',
 '##ar',
 '##cha',
 'ni',
 '##fe',
 '##e',
 '##9',
 'w',
 '7',
 '##etta',
 'li',
 '3',
 '##om',
 '##rou',
 '##u',
 'sn',
 '##in',
 'wala',
 'ye',
 '##f',
 '##hem',
 '##m',
 'ma',
 '##5',
 '##ye',
 '##b',
 '##kom',
 '##m']

In [24]:
def tokenize_sent_max(sentence,max_len=512):
    token=tokenizer.tokenize(sentence)
    token=token[:max_len-2]
    return token

In [34]:

#creating the field object
TEXT=ttd.Field(#sequential=True,
               batch_first=True,
               pad_first=True,
               tokenize=tokenize_sent_max,
               preprocessing=tokenizer.convert_tokens_to_ids,
               init_token=tokenizer.cls_token_id,
               eos_token=tokenizer.sep_token_id,
               unk_token=tokenizer.unk_token_id,
               pad_token=tokenizer.pad_token_id,
               use_vocab=False,
               is_target=False)
LABEL=ttd.Field(sequential=False,is_target=True,use_vocab=False)

In [35]:
dataset=ttd.TabularDataset(path='../ppd/Train.csv',
                   format='csv',
                    skip_header=True,
                   fields=[('text',TEXT),('label',LABEL)])

In [38]:
tokenizer.max_model_input_sizes


{'bert-base-uncased': 512,
 'bert-large-uncased': 512,
 'bert-base-cased': 512,
 'bert-large-cased': 512,
 'bert-base-multilingual-uncased': 512,
 'bert-base-multilingual-cased': 512,
 'bert-base-chinese': 512,
 'bert-base-german-cased': 512,
 'bert-large-uncased-whole-word-masking': 512,
 'bert-large-cased-whole-word-masking': 512,
 'bert-large-uncased-whole-word-masking-finetuned-squad': 512,
 'bert-large-cased-whole-word-masking-finetuned-squad': 512,
 'bert-base-cased-finetuned-mrpc': 512,
 'bert-base-german-dbmdz-cased': 512,
 'bert-base-german-dbmdz-uncased': 512,
 'TurkuNLP/bert-base-finnish-cased-v1': 512,
 'TurkuNLP/bert-base-finnish-uncased-v1': 512,
 'wietsedv/bert-base-dutch-cased': 512}

In [39]:
ex=dataset.examples[1]

In [40]:
ex.text

[16623,
 10959,
 20651,
 14908,
 10158,
 11518,
 10624,
 14795,
 10150,
 10562,
 45743,
 10131,
 10546,
 11444,
 14519,
 71816,
 10391,
 15966,
 22523,
 10116]

In [41]:
train,test=dataset.split(0.8)

In [42]:
train_iter,test_iter=ttd.Iterator.splits(datasets=(train,test),batch_sizes=(32,256),sort_key=lambda x: len(x.text))

In [43]:
test_iter

In [46]:
for inputs,target in test_iter:
    print(inputs.shape)
    print(target.shape)
    break

torch.Size([256, 5])
torch.Size([256])


In [3]:
a=list(zip(*('jamiu','muhammed')))

In [4]:
a

[('j', 'm'), ('a', 'u'), ('m', 'h'), ('i', 'a'), ('u', 'm')]

In [1]:
def customPadding(batch,tokenizer=E.tokenizer):
    comp=list(zip(*batch))
    tokens=comp[0]
    masks=comp[1]
    len_tokens=comp[2]
    labels=comp[3]
    max_len=max(len_tokens)
    tokens_ret=[]
    masks_ret=[]
    
    for idx in range(len(tokens)):
        pad_len=max_len-min(len_tokens[idx],max_len)
        padding=[tokenizer.pad_token_id] * pad_len
        token=tokens[idx] + padding
        mask=masks[idx] + [0] * pad_len
        tokens_ret.append(token)
        masks_ret.append(mask)
        
    if len(comp)==4:
        return torch.tensor(tokens_ret),torch.tensor(masks_ret),torch.tensor(labels)
    return torch.tensor(tokens_ret),torch.tensor(masks_ret)

        

NameError: name 'E' is not defined

In [ ]:
def customPadding(batch,tokenizer=E.tokenizer):
    comp=list(zip(*batch))
    tokens=comp[0]
    masks=comp[1]
    len_tokens=comp[2]
    labels=comp[3]
    max_len=max(len_tokens)
    tokens_ret=np.zeros((len(batch), max_len), dtype=np.int64)
    masks_ret=np.zeros((len(batch), max_len), dtype=np.int64)
    
    for i, row in enumerate(tokens):
        row = np.array(row[:max_len])
        tokens_ret[i, :len(row)] = row
    token_tensor = torch.from_numpy(tokens_ret)
    
    for i, row in enumerate(masks):
        row = np.array(row[:max_len])
        masks_ret[i, :len(row)] = row
    mask_tensor = torch.from_numpy(masks_ret)
    
    """for idx in range(len(tokens)):
        pad_len=max_len-min(len_tokens[idx],max_len)
        padding=[tokenizer.pad_token_id] * pad_len
        token=tokens[idx] + padding
        mask=masks[idx] + [0] * pad_len
        tokens_ret.append(token)
        masks_ret.append(mask)"""
        
    if len(comp)==4:
        return token_tensor,mask_tensor,torch.tensor(labels)
    return token_tensor,mask_tensor
        